# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [ ]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.84MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [ ]:
#we have another way

from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='/content/Dataset.csv' )
data = loader.load()

In [ ]:
data

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [ ]:
markdown_violations = []

for index, row in df.iterrows():
    violation = row['المخالفة']
    fine = row['الغرامة']
    markdown_text = f"المخالفة: {violation} - الغرامة: {fine}."
    markdown_violations.append(markdown_text)

In [ ]:
for entry in markdown_violations[:5]:
    print(entry)

المخالفة: قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة: الغرامة المالية 100 - 150 ريال.
المخالفة: ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. - الغرامة: الغرامة المالية 100 - 150 ريال.
المخالفة: عدم وجود تأمين ساري للمركبة. - الغرامة: الغرامة المالية 100 - 150 ريال.
المخالفة: عبور المشاة للطرق من غير الأماكن المخصصة لهم. - الغرامة: الغرامة المالية 100 - 150 ريال.
المخالفة: عدم تقيد المشاة بالإشارات الخاصة بهم. - الغرامة: الغرامة المالية 100 - 150 ريال.


## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [ ]:
! pip install langchain_community

  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tena

In [ ]:
!pip install langchain

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 512
splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size)

chunks = splitter.create_documents(markdown_violations)

In [ ]:
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 104


## Step 5: Generate Embeddings for the Documents

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 6.9 MB/s eta 0:00:00


Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="asafaya/bert-base-arabic")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
!pip install langchain_chroma

  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.6 MB/s eta 0:00:00
   ━━

In [ ]:
!pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template= "   لاتجب على الاسئلة الاخرى ،فيما يلي قائمة بالمخالفات المرورية والغرامات   : \n {context} \n \n بناءً على القائمة أعلاه، أجب على السؤال التالي: {question}."
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
!pip install langchain-groq

  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
groq_api_key = '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'

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7,
    api_key=groq_api_key
)

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [ ]:
from langchain import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
def query_rag(question):

    relevant_context = "..."

    response = chain.run({
        "context": relevant_context,
        "question": question
    })

    return response

In [ ]:
#we will try it

user_query = "ماهي الغرامة على القيادة بدون رخصة؟"

answer = query_rag(user_query)
print(answer)

<ipython-input-25-d66978979bf0>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run({


لا أرى القائمة في أسفل السؤال. ولكن يمكنني إجابة السؤال برغم ذلك.

غرامة القيادة بدون رخصة تختلف من دولة إلى أخرى، ولكن العامة تتراوح بين 500 إلى 2000 دولار أو حتى إيقاف السيارة وتأهيلها لاتخاذ الإجراءات القانونية.


In [ ]:
#we will try it

user_query = 'ماهي الغرامة عدم تقيد المشاة بالإشارات الخاصة بهم.'

answer = query_rag(user_query)
print(answer)

حسناً. على أساس القائمة في سؤالك، الغرامة عدم تقيد المشاة بالإشارات الخاصة بهم هي: 150.


when we use `query_rag` not work will so we try another way



---
## other way

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k":3})

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever=retriever,
    return_source_documents=False,
)

In [ ]:
query = "ما هي الغرامة عدم تقيد المشاة بالإشارات الخاصة به"
result = qa_chain(query)
print(f'Answer:', {result['result']})

Answer: {'الغرامة المالية لعدم تقيد المشاة بالإشارات الخاصة بهم هي 100-150 ريال.'}


In [ ]:
query = "طريقة عمل البيتزا"
result = qa_chain(query)
print(f'Answer:', {result['result']})

Answer: {'لا يوجد علاقة بين السؤال الخاص بك و السيارة التي ذكرتها في السياق الذي قدمته.'}


In [ ]:
query = "هل هناك مخالفة على المركبة بدون لوحة وكم سعر المخالفة"
result = qa_chain(query)
print(f'Answer:', {result['result']})

Answer: {'نعم، هناك مخالفة على المركبة بدون لوحة خلفية. المخالفة هي: سير المركبة بلا لوحة خلفية، أو بلا لوحات. الغرامة المالية لذلك هي 3000 - 6000 ريال.'}


In [ ]:
query = "غرامة عدم حمل رخصة"
result = qa_chain(query)
print(f'Answer:', {result['result']})

Answer: {'غرامة عدم حمل رخصة القيادة هي 1000 - 2000 ريال.'}


In [ ]:
query = "غرامة عدم حمل رخصة"
result = qa_chain(query)
print(f'Answer:', {result['result']})

Answer: {'بالتأكيد، وفقًا للبيانات المتاحة علي وقع المخالفة "قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة" الغرامة 1000 - 2000 ريال.'}


In [ ]:
query = "حمل رخصة"
result = qa_chain(query)
print(f'Answer:', {result['result']})

Answer: {'هل أنت تريد سؤال عن الغرامة للعامل في موضوع رخصة السواقه؟\n\nسيتم الإجابة على الذي ذكرت أو أى سؤال يتعلق بحقائب رخص شهر الشوال خاصنك، أو جبت لتقصر على شك  سؤالك غير مدروس'}


In [ ]:
query = "غرامة إساءة استعمال منبة المركبة"
result = qa_chain(query)
print(f'Answer:', {result['result']})

Answer: {'الغرامة المالية لهذه المخالفة هي 150 - 300 ريال.'}




---



## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

Future Work:

1. **Expanding the Database:**
   - **Adding more data:** You may need to include more traffic violations and fines from multiple sources, especially from different countries or traffic laws, to make the system more comprehensive.
   - **Regular data updates:** Traffic laws and fines change over time, so the data should be updated regularly to ensure the accuracy of the responses.

2. **Improving the Quality of Numerical Representations (Embeddings):**
   - **Using advanced language models:** Retrieval accuracy can be improved by using more advanced models such as OpenAI GPT-4 or Hugging Face Arabic Transformer, which can be more accurate in understanding legal language and specific terms related to violations.
   - **Fine-tuning models:** You can train the models specifically on traffic data to make them more capable of understanding questions related to violations and fines.

In [ ]:
import pandas as pd

# تحميل البيانات (كما تم بالفعل)
df = pd.read_csv('/content/Dataset.csv')

# دالة للبحث عن المخالفة
def search_violation(question):
    # البحث عن المخالفة في النص
    for index, row in df.iterrows():
        if row['المخالفة'] in question:
            return f"{row['المخالفة']} - {row['الغرامة']}"
    return "السؤال غير موجود في قاعدة البيانات. الرجاء إدخال سؤال يتعلق بالمخالفات."

# اختبار دالة البحث
question = input("أدخل سؤالك هنا: ")
answer = search_violation(question)
print(answer)

KeyboardInterrupt: Interrupted by user